In [1]:
# Step 1: find the GM letters in a bigger archive of the Globalise project
# using https://hdl.handle.net/10622/BHKMWE

In [2]:
DATA_PREFIX = '/ivi/ilps/personal/vprovat/GM_VOC/'

import pandas as pd

gm_df = pd.read_csv(DATA_PREFIX+'overzicht_GM.csv',sep=';',dtype='str')

In [2]:
% # remove/comment this line to see one entry of the overview table
gm_df.iloc[0]

ID                                                                                              1
Problemen gevonden tijdens handmatige check:                                                  NaN
ID in TANAP database                                                                          908
Inv.nr. Nationaal Archief (1.04.02)                                                          1068
Deel v. inventarisnummer                                                                      NaN
Kamer                                                                                   Amsterdam
Beginfolio                                                                                    275
Eindfolio                                                                                     278
Beginscan                                                                                     611
Eindscan                                                                                      617
Enkele of dubbele pa

In [3]:
import numpy as np

In [4]:
# to know which text files are GMs and discard all other ones (no way to know in advance)
gm_filenames = set([str(int(name)) for name in 
                       gm_df['Inv.nr. Nationaal Archief (1.04.02)'].dropna().tolist()
                   ])
# gm_filenames

In [5]:
% # remove this line if running for the first time: installing wget to download files
import sys
!{sys.executable} -m pip install python3-wget

UsageError: Line magic function `%` not found.


In [6]:
% # remove this line if running for the first time: downloading the text files
import wget
import os
from tqdm.auto import tqdm
for line in tqdm(open(DATA_PREFIX+'globalise_transcriptions_v2_txt.tab','r').readlines()):
    filename_raw = wget.download(line.strip(),out=DATA_PREFIX[:-1])
    filename = filename_raw.split('/')[-1].split('.')[0]
    if filename not in gm_filenames: # some text files are not GMs
        os.remove(filename_raw)

UsageError: Line magic function `%` not found.


In [7]:
% # remove this line if you want to see what's inside a file
test_filename = '7957.txt'
with open(DATA_PREFIX+test_filename,'r') as f:
    text = f.read()
for line in text.split('\n'):
    print(line)

UsageError: Line magic function `%` not found.


In [8]:
import re

text_per_page = {} # scan name: text

def split_into_pages(raw_text):
    '''example:
        #+ ---
        #+ NL-HaNA_1.04.02_1068_0004.xml
        #+ ---
        TEXT HERE
    '''
    page_name_start = raw_text.find('#+ ---') 
    if page_name_start == -1: # no more pages here
        return
    page_name_start += len('#+ ---\n#+ ')
    page_name_end = raw_text[page_name_start:].find('.xml') + page_name_start
    page_name = raw_text[page_name_start:page_name_end]
    page_text_start = page_name_end + len('.xml\n#+ ---\n')
    page_text_end = raw_text[page_text_start:].find('#+ ---') # next page begins here
    if page_text_end == -1: # this was the last page
        page_text_end = len(raw_text)
    else:
        page_text_end += page_text_start
    page_text = raw_text[page_text_start:page_text_end]
    
    text_per_page[page_name] = page_text
    return split_into_pages(raw_text[page_text_end:])

In [9]:
% #remove this line to test on the sample file from above
split_into_pages(text)

UsageError: Line magic function `%` not found.


In [13]:
from tqdm.auto import tqdm


# We've downloaded all files but maybe some GMs from the table are missing, let's see which ones
rows_missing_text = []
for filename in tqdm(gm_filenames):
    try:
        with open(DATA_PREFIX+filename+'.txt','r') as f:
            text = f.read()
            split_into_pages(text)
    except:
        print('MISSING FILE: ',filename)
        rows = gm_df[gm_df['Inv.nr. Nationaal Archief (1.04.02)']==filename]
        rows_missing_text.append(rows)

  0%|          | 0/540 [00:00<?, ?it/s]

MISSING FILE:  2770
MISSING FILE:  1179
MISSING FILE:  1887
MISSING FILE:  1171
MISSING FILE:  1457
MISSING FILE:  2911
MISSING FILE:  1684


In [22]:
df_missing_text = pd.concat(rows_missing_text)
df_missing_text.to_csv('missing_gms.tsv',sep='\t')
df_missing_text

,ID,Problemen gevonden tijdens handmatige check:,ID in TANAP database,Inv.nr. Nationaal Archief (1.04.02),Deel v. inventarisnummer,Kamer,Beginfolio,Eindfolio,Beginscan,Eindscan,...,Vroegste jaar,Laatste jaar,Beschrijving in TANAP,Datum,Datum (numeriek),LINK naar eerste scan,Bestandsnaam van eerste scan,LINK naar laatste scan,Bestandsnaam van laatste scan,HTR van IJsberg beschikbaar?
639,640,Niet gedigitaliseerd.,71958,2770,NaN,Amsterdam,726,742,1466,1498,...,1751,1751,Origineele generale missive door den gouverneu...,31 December 1751 [aan hunne hoog mogenden over...,1751-12-31,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_2770_1466,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_2770_1498,FALSE
640,641,Niet gedigitaliseerd.,71975,2770,NaN,Amsterdam,890,902,1794,1818,...,1752,1752,Een origineele missive door generaal en raaden...,31 Maart 1752.,1752-03-31,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_2770_1794,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_2770_1818,FALSE
641,642,Niet gedigitaliseerd.,71983,2770,NaN,Amsterdam,931,940,1876,1894,...,1752,1752,Origineele missive door generaal en raaden ges...,6 Junij 1752.,1752-06-06,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_2770_1876,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_2770_1894,FALSE
117,118,Wegens beschadiging niet raadpleegbaar.,9427,1179,A,Amsterdam,1,155,NaN,NaN,...,1650,1650,Originele generale missive door den gouverneur...,10 December 1650 [NB. fls. 1-36 (wegens onlees...,1650-12-10,NaN,NaN,NaN,NaN,FALSE
118,119,Moderne kopie van missive 118. Folio 1-36 en 1...,9459,1179,B,Amsterdam,1,155,5,324,...,1650,1650,Originele generale missive door den gouverneur...,10 December 1650 [NB. fls. 1-36 (wegens onlees...,1650-12-10,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1179B_0005,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1179B_0324,FALSE
411,412,NaN,NaN,1887,NaN,Amsterdam,14,600,39,1214,...,1721,1721,NaN,30 november 1717,1717-11-30,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1887_0039,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1887_1214,FALSE
113,114,Niet gedigitaliseerd.,9131,1171,NaN,Amsterdam,1,50,1,99,...,1650,1650,"Gedeelte van een generale missive, ongedateerd.",ongedateerd,NaN,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1171_0001,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1171_0099,FALSE
281,282,NaN,33112,1457,A,Amsterdam,11,537,37,1077,...,1689,1689,Originele generale missive van den gouverneur ...,30 December 1689.,1689-12-30,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1457A_0037,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1457A_1077,FALSE
282,283,NaN,33113,1457,A,Amsterdam,538,541,1081,1088,...,1689,1689,Originele missive van den gouverneur generael ...,30 November 1689.,1689-11-30,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1457A_1081,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1457A_1088,FALSE
283,284,NaN,33114,1457,A,Amsterdam,542,545,1089,1095,...,1689,1689,Originele missive in authenticque copie van de...,30 November 1689.,1689-11-30,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1457A_1089,https://www.nationaalarchief.nl/onderzoeken/ar...,NL-HaNA_1.04.02_1457A_1095,FALSE


In [19]:
page_names = set(gm_df['Bestandsnaam van eerste scan'].tolist()+gm_df['Bestandsnaam van laatste scan'].tolist())
missing_page_names = page_names - set(text_per_page.keys())

In [37]:
missing_page_names_problematic = []
missing_doc_ids = set(df_missing_text['Inv.nr. Nationaal Archief (1.04.02)'].to_list())
for page_name in missing_page_names:
    if not isinstance(page_name, str):
        continue
    page_id = page_name.split('_')[-2]
    if page_id not in missing_doc_ids and page_id[:-1] not in missing_doc_ids:
        missing_page_names_problematic.append(page_name)
missing_page_names_problematic

['NL-HaNA_1.04.02_1068_617',
 'NL-HaNA_1.04.02_7957_',
 'NL-HaNA_1.04.02_1212_0009',
 'NL-HaNA_1.04.02_1212_0204']

In [61]:
# correcting the annotation problems manually:

text_per_page['NL-HaNA_1.04.02_1068_617'] = text_per_page['NL-HaNA_1.04.02_1068_0617'] 

gm_df['NA_ID'] = gm_df['Inv.nr. Nationaal Archief (1.04.02)'] # to use the dot notation, not possible with spaces

gm_df.loc[gm_df.NA_ID=='1212','Bestandsnaam van eerste scan'] = 'NL-HaNA_1.04.02_1212_1001' 
# Column w. correct annotation: Beginscan
gm_df.loc[gm_df.NA_ID=='1212','Bestandsnaam van laatste scan'] = 'NL-HaNA_1.04.02_1212_1196'
# Column w. correct annotation: Eindscan

gm_df.loc[gm_df.NA_ID=='7957','Bestandsnaam van eerste scan'] = 'NL-HaNA_1.04.02_7957_0001'
gm_df.loc[gm_df.NA_ID=='7957','Bestandsnaam van laatste scan'] = 'NL-HaNA_1.04.02_7957_0004'

In [62]:
import numpy as np
def get_text(row):
    text = ''
    if pd.isna(row['Beginscan']):
        return np.nan
    first_page_name = row['Bestandsnaam van eerste scan']
    last_page_name = row['Bestandsnaam van laatste scan']
    first_page_number = int(row['Beginscan'])
    last_page_number = int(row['Eindscan'])
    if not isinstance(last_page_name,str) or not isinstance(first_page_name,str):
        print('Misssing scan')
        return np.nan
    page_name_prefix = '_'.join(first_page_name.split('_')[:-1]+[''])
#     first_page_number = int(first_page_name.split('_')[-1])
#     last_page_number = int(last_page_name.split('_')[-1])
    for cnt in range(first_page_number, last_page_number+1):
        page_name_no_zero = page_name_prefix + str(cnt) # sometimes it's eg. _617
        if page_name_no_zero in text_per_page:
            text += text_per_page[page_name_no_zero] + '\n\n'
        else:
            page_name_with_zero = page_name_prefix + '0'*(4-len(str(cnt))) + str(cnt) # sometimes _0611
            if page_name_with_zero not in text_per_page:
                print('MISSING PAGE: ',page_name_with_zero)
            else:
                text += text_per_page[page_name_with_zero]
    return text

In [63]:
gm_df['Tekst'] = gm_df.apply(get_text,axis=1)

MISSING PAGE:  NL-HaNA_1.04.02_1171_0001
MISSING PAGE:  NL-HaNA_1.04.02_1171_0002
MISSING PAGE:  NL-HaNA_1.04.02_1171_0003
MISSING PAGE:  NL-HaNA_1.04.02_1171_0004
MISSING PAGE:  NL-HaNA_1.04.02_1171_0005
MISSING PAGE:  NL-HaNA_1.04.02_1171_0006
MISSING PAGE:  NL-HaNA_1.04.02_1171_0007
MISSING PAGE:  NL-HaNA_1.04.02_1171_0008
MISSING PAGE:  NL-HaNA_1.04.02_1171_0009
MISSING PAGE:  NL-HaNA_1.04.02_1171_0010
MISSING PAGE:  NL-HaNA_1.04.02_1171_0011
MISSING PAGE:  NL-HaNA_1.04.02_1171_0012
MISSING PAGE:  NL-HaNA_1.04.02_1171_0013
MISSING PAGE:  NL-HaNA_1.04.02_1171_0014
MISSING PAGE:  NL-HaNA_1.04.02_1171_0015
MISSING PAGE:  NL-HaNA_1.04.02_1171_0016
MISSING PAGE:  NL-HaNA_1.04.02_1171_0017
MISSING PAGE:  NL-HaNA_1.04.02_1171_0018
MISSING PAGE:  NL-HaNA_1.04.02_1171_0019
MISSING PAGE:  NL-HaNA_1.04.02_1171_0020
MISSING PAGE:  NL-HaNA_1.04.02_1171_0021
MISSING PAGE:  NL-HaNA_1.04.02_1171_0022
MISSING PAGE:  NL-HaNA_1.04.02_1171_0023
MISSING PAGE:  NL-HaNA_1.04.02_1171_0024
MISSING PAGE:  N

MISSING PAGE:  NL-HaNA_1.04.02_1457A_0037
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0038
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0039
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0040
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0041
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0042
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0043
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0044
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0045
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0046
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0047
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0048
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0049
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0050
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0051
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0052
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0053
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0054
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0055
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0056
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0057
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0058
MISSING PAGE:  NL-HaNA_1.04.02_1457A_0059
MISSING PAGE:  NL-HaNA_1.04.02_145

MISSING PAGE:  NL-HaNA_1.04.02_1457A_1088
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1089
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1090
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1091
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1092
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1093
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1094
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1095
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1097
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1098
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1099
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1100
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1101
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1102
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1103
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1104
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1105
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1106
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1107
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1108
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1109
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1110
MISSING PAGE:  NL-HaNA_1.04.02_1457A_1111
MISSING PAGE:  NL-HaNA_1.04.02_145

MISSING PAGE:  NL-HaNA_1.04.02_1887_1117
MISSING PAGE:  NL-HaNA_1.04.02_1887_1118
MISSING PAGE:  NL-HaNA_1.04.02_1887_1119
MISSING PAGE:  NL-HaNA_1.04.02_1887_1120
MISSING PAGE:  NL-HaNA_1.04.02_1887_1121
MISSING PAGE:  NL-HaNA_1.04.02_1887_1122
MISSING PAGE:  NL-HaNA_1.04.02_1887_1123
MISSING PAGE:  NL-HaNA_1.04.02_1887_1124
MISSING PAGE:  NL-HaNA_1.04.02_1887_1125
MISSING PAGE:  NL-HaNA_1.04.02_1887_1126
MISSING PAGE:  NL-HaNA_1.04.02_1887_1127
MISSING PAGE:  NL-HaNA_1.04.02_1887_1128
MISSING PAGE:  NL-HaNA_1.04.02_1887_1129
MISSING PAGE:  NL-HaNA_1.04.02_1887_1130
MISSING PAGE:  NL-HaNA_1.04.02_1887_1131
MISSING PAGE:  NL-HaNA_1.04.02_1887_1132
MISSING PAGE:  NL-HaNA_1.04.02_1887_1133
MISSING PAGE:  NL-HaNA_1.04.02_1887_1134
MISSING PAGE:  NL-HaNA_1.04.02_1887_1135
MISSING PAGE:  NL-HaNA_1.04.02_1887_1136
MISSING PAGE:  NL-HaNA_1.04.02_1887_1137
MISSING PAGE:  NL-HaNA_1.04.02_1887_1138
MISSING PAGE:  NL-HaNA_1.04.02_1887_1139
MISSING PAGE:  NL-HaNA_1.04.02_1887_1140
MISSING PAGE:  N

Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan
Misssing scan


In [68]:
gm_df.to_csv(DATA_PREFIX+'overzicht_GM_met_teksten.csv',sep=';')

In [11]:
% # remove this line if continuing to run from here
gm_df = pd.read_csv(DATA_PREFIX+'overzicht_GM_met_teksten.csv',sep=';')

UsageError: Line magic function `%` not found.


In [ ]:
# Now, converting to the same format as the previous dataframe in our experiments

In [71]:
gm_df.columns

Index(['ID', 'Problemen gevonden tijdens handmatige check:',
       'ID in TANAP database', 'Inv.nr. Nationaal Archief (1.04.02)',
       'Deel v. inventarisnummer', 'Kamer', 'Beginfolio', 'Eindfolio',
       'Beginscan', 'Eindscan', 'Enkele of dubbele pagina op scan',
       'RGP Deel waarin de missive is opgenomen',
       'RGP pagina waarop de missive begint', 'Jaren', 'Vroegste jaar',
       'Laatste jaar', 'Beschrijving in TANAP', 'Datum', 'Datum (numeriek)',
       'LINK naar eerste scan', 'Bestandsnaam van eerste scan',
       'LINK naar laatste scan', 'Bestandsnaam van laatste scan',
       'HTR van IJsberg beschikbaar?', 'NA_ID', 'Tekst'],
      dtype='object')

In [12]:
df_letters_all = pd.DataFrame() # empty dataframe to be later used for our experiments (to preserve the format)

In [14]:
gm_df_dated = gm_df.dropna(subset=['Datum (numeriek)'])
set(gm_df_dated['Datum (numeriek)'].to_list())

{'1618-06-24',
 '1618-07-26',
 '1618-09-29',
 '1618-10-05',
 '1618-11-12',
 '1619-08-05',
 '1619-10-07/1619-10-15',
 '1620-01-22',
 '1620-05-11',
 '1620-07-31',
 '1620-10-26',
 '1621-01-08',
 '1621-07-09',
 '1621-11-16',
 '1621-12-20',
 '1622-01-21',
 '1622-03-14',
 '1622-03-26',
 '1622-09-06',
 '1623-02-24/1623-12-25',
 '1623-12-25',
 '1624-01-03',
 '1625-01-27',
 '1625-08-13/1625-09-13',
 '1626-02-03',
 '1626-02-15',
 '1626-12-13',
 '1626-12-15',
 '1626-12-26',
 '1627-03-04',
 '1627-11-09',
 '1628-01-06',
 '1628-01-06/1628-01-13',
 '1628-01-13',
 '1628-01-24',
 '1628-11-03',
 '1628-11-17',
 '1629-03-18',
 '1629-12-15',
 '1631-03-07',
 '1632-01-06',
 '1632-12-01',
 '1633-02-07',
 '1633-02-08',
 '1633-08-15',
 '1633-12-15/1633-12-25',
 '1634-08-15',
 '1635-01-08',
 '1635-12-31',
 '1636-01-04',
 '1636-02-02',
 '1636-12-28',
 '1637-12-09',
 '1638-12-22',
 '1639-01-10',
 '1639-01-12',
 '1639-12-18',
 '1640-01-08',
 '1640-01-31',
 '1640-09-09',
 '1640-11-30',
 '1641-01-08',
 '1641-01-31',


In [17]:
'1633-12-15/1633-12-25'.split('/')

['1633-12-15', '1633-12-25']

In [18]:
df_letters_all['rawdate'] = gm_df_dated['Datum']
df_letters_all['year'] = gm_df_dated['Datum (numeriek)'].apply(lambda x: x.split('/')[0].split('-')[0]) # first date in the interval
df_letters_all['month'] = gm_df_dated['Datum (numeriek)'].apply(lambda x: x.split('/')[0].split('-')[1])
df_letters_all['day'] = gm_df_dated['Datum (numeriek)'].apply(lambda x: x.split('/')[0].split('-')[2] 
                                                              if len(x.split('/')[0].split('-'))>2 else 1) # if day is missing, use first
df_letters_all['text'] = gm_df_dated['Tekst']

In [19]:
df_letters_all

,rawdate,year,month,day,text
0,5 October 1618.,1618,10,05,"\n275\nDrentfeste. wijse, voorsienige heer dis..."
1,12 November 1618.,1618,11,12,\n264\nErntfestel wijse Voorsinnige seer Ducre...
2,29 September 1618.,1618,09,29,\n268\nCopia.\nubbe\npatria. aen H=ren Maijore...
3,26 Julij 1618 [tot Jaccattra].,1618,07,26,\nria ten Tholen\n21\n80\nErntfeste wijse Voor...
4,24 Junij 1618 [tot Jaccatra].,1618,06,24,\n63\ntrian P den W. Beer.\nErntfeste wijse vo...
...,...,...,...,...,...
918,den 17 April 1792.,1792,04,17,\nPatria\n51.\naanbrengen\nder brief van\n19 A...
919,den 25 Maij anno 1792.,1792,05,25,\nDupl:\nZ. d\nFatria\nAan de WelEdele Hoog Ac...
920,den 15 Junij 1792.,1792,06,15,\nNo 15\nGemeene\nZ. D.\nPatria\nAgtbaare Heer...
921,den 27 Junij 1792.,1792,06,27,\ngemeen\nPatria\n87\nAan de wel Edele Hoog Ag...


In [21]:
df_letters_all.to_csv('data/letters_latest.tsv',sep='\t')

In [3]:
# removing duplicates

df_letters_all = pd.read_csv('data/letters_latest.tsv',sep='\t')

In [10]:
df_letters_all[df_letters_all.duplicated('text')]

,Unnamed: 0,rawdate,year,month,day,text
116,118,10 December 1650 [NB. fls. 1-36 (wegens onlees...,1650,12,10,NaN
199,206,1 April 1655.,1655,4,1,"\n487\nPatria aende.\nHeeren Zeventiene,\nNa d..."
200,207,12 Julij 1655.,1655,7,12,\n538\nis\nEede Erntfeste wijse —\nvoorsienige...
214,221,5 julij 1677,1677,7,5,NaN
274,281,30 December 1689.,1689,12,30,NaN
275,282,30 November 1689.,1689,11,30,NaN
276,283,30 November 1689.,1689,11,30,NaN
277,284,30 December 1689.,1689,12,30,NaN
344,351,31 januari 1705,1705,1,31,NaN
345,352,17 februari 1705,1705,2,17,NaN
